# Symbols

Symbols are a primitive in Javascript. The only way to create a symbol is via the class constructor `Symbol`, which creates a unique symbol every time it is invoked.

In [ ]:
var a = Symbol()
var b = Symbol()

console.log(`a: ${a.toString()}`)
console.log(`a == b: ${a == b}`)

Unlike other primitives, there is no wrapper type for symbols. This was to avoid confusion, since the constructor creates a unique symbol every time.

In [ ]:
new Symbol()

The only intended purpose of a symbol is to be used as an object's property.

In [ ]:
obj = {}
obj["provider_id"] = 100
obj[Symbol()] = "secret property?"

In [ ]:
Object.getOwnPropertyNames(obj)

In [ ]:
obj.provider_id

It's not possible to access the secret property by creating a symbol, since the new symbol will not equal the symbol used as the key.

In [ ]:
obj[Symbol()]

However, we can still get to it with a little bit of reflection.

In [ ]:
Object.getOwnPropertySymbols(obj)

In [ ]:
secret_key = Object.getOwnPropertySymbols(obj)[0]
obj[secret_key]

Let's use this approach to see how we could get "private" methods.

In [ ]:
"use strict"
var validLocations = Symbol()
var Locations = class {
    constructor(locations) {
        this.locations = locations
    }
       
    getIds() {
        let locs = this[validLocations]()
        return locs.map((loc) => {return loc.id})
    }
    
    [validLocations]() {
        return this.locations.filter((loc) => {return loc.id != undefined && loc.id != null})
    }
}
var c = new Locations([{id: 1}, {id: null}, {id: 10}])

In [ ]:
c.getIds()

In [ ]:
Object.getOwnPropertyNames(c.__proto__)

In [ ]:
Object.getOwnPropertySymbols(c.__proto__)

In [ ]:
var private_method = Object.getOwnPropertySymbols(c.__proto__)[0]
c[private_method]()

## Enums?
The semantics are cumbersome, but Symbols can also be used to implement a basic enum.

In [ ]:
'use strict'
var Suits = {
    HEARTS: Symbol("Hearts"),
    DIAMONDS: Symbol("Diamonds"),
    CLUBS: Symbol("Clubs"),
    SPADES: Symbol("Spades"),
    show: (suit) => {
        // There is no method for retrieving a string from a symbol. Here's a workaround that leverages toString()
        var pattern = /^Symbol\((.+)\)$/
        return suit.toString().match(pattern)[1]
    }
}

var Card = class {
    constructor(rank, suit) {
        this.rank = String(rank)
        this.suit = suit
    }
    
    toString() {
        return `${this.rank} of ${Suits.show(this.suit)}`
    }
}

In [ ]:
c = new Card(1, Suits.HEARTS)
c.toString()

## Well Known Symbols
Well-known symbols are special symbols exposed by javascript for developers to implement. They are similar to Python's dunder ("magic") methods.
Let's look at the well-known symbol `Symbol.iterator`, which can be thought of as an analog to Python's `__iter__`

In [ ]:
'use strict'
var Deck = class {
    constructor() {
        this.cards = []
        for (let suit of [Suits.HEARTS, Suits.DIAMONDS, Suits.SPADES, Suits.CLUBS]) {
            for (let i=1;i<14;i++) {
                this.cards.push(new Card(i, suit))
            }
        }
    }
    
    [Symbol.iterator]() {
        return this.cards[Symbol.iterator]()
    }
}

In [ ]:
'use strict'
var deck = new Deck()
for (let card of deck) {
    if (card.suit == Suits.SPADES) {
        console.log(card.toString())
    }
}

# Useful Sources

* MDN [Entry](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Symbol) and [Glossary Entry](https://developer.mozilla.org/en-US/docs/Glossary/Symbol) for Symbols
* [Exploringjs Chapter on Symbols](http://exploringjs.com/es6/ch_symbols.html) - An in-depth look at symbols and how they behave, with many examples.

# Down the Rabbit Hole
While making this I bounced around several different ideas for a simple enum implementation using symbols. I picked a simple one for the sake of presentation, but there is an annoying limitation: you can't iterate over it without bringing in `show`. There are two ideas I thought of that would mitigate this, neither of which are aesthetically pleasing.

### Using the prototype
Putting `show` on the prototype of `Suits` would prevent it from being caught by `Object.keys()`

In [ ]:
'use strict'
var Suits = {
    HEARTS: Symbol("Hearts"),
    DIAMONDS: Symbol("Diamonds"),
    CLUBS: Symbol("Clubs"),
    SPADES: Symbol("Spades")
}
Suits.__proto__ = {
    show: (suit) => {
        var pattern = /^Symbol\((.+)\)$/
        return suit.toString().match(pattern)[1]
    }
}

In [ ]:
Suits.show(Symbol('123'))

In [ ]:
Object.keys(Suits)

### Using Property Descriptors
We can modify the property descriptor of `toString` to make it not enumerable.

In [ ]:
var Suits = {
    HEARTS: Symbol("Hearts"),
    DIAMONDS: Symbol("Diamonds"),
    CLUBS: Symbol("Clubs"),
    SPADES: Symbol("Spades"),
    show: (suit) => {
        // There is no method for retrieving a string from a symbol. Here's a workaround that leverages toString()
        var pattern = /^Symbol\((.+)\)$/
        return suit.toString().match(pattern)[1]
    }
}
Object.defineProperty(Suits, 'show', {enumerable: false})

Object.keys(Suits)

## Wait, Why?
At this point, I wondered why use symbols at all. We could have just created an object that mapped all-caps keys to strings, and we wouldn't even need a tricky `show` function in the first place!

### Type Safety
One of the primary advantages of enums touted in other languages is type safety. Using symbols ensures that the only value that can ever equal a member of an enum is the member itself. This eliminates a whole class of type-related bugs (e.g. we attempt to do equality checks with strings and misspell one.)

### Readability
A consequence of type safety, enums implemented with symbols makes code more readable since the only way to fetch a member is from the enum itself. `Suits.HEARTS` contains all the context an engineer needs to understand what this value is and why it is being used. `"hearts"` is a little more cryptic, and exposes a host of methods that don't make sense for a card suit (e.g. string methods like `search()` and `substring()`). The fact that symbols are so limited is also their strength.

If we are serious about type safety, then we should also be able to determine whether a symbol belongs to an enumeration. Luckily, we can use the well-known symbol `Symbol.hasInstance` to do exactly this!

In [3]:
'use strict'
var Suits = {
    HEARTS: Symbol("Hearts"),
    DIAMONDS: Symbol("Diamonds"),
    CLUBS: Symbol("Clubs"),
    SPADES: Symbol("Spades"),
    [Symbol.hasInstance]: (instance) => {
        return Object.values(Suits).indexOf(instance) > -1
    }
}

Suits.HEARTS instanceof Suits

true